In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import pandas as pd
import csv
import tensorflow as tf
import nltk 
from tensorflow.contrib.rnn import RNNCell, LSTMStateTuple
import gensim as gs

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.contrib import rnn

from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

nltk.download('punkt')

df1 = pd.read_csv('HillaryTrumpTrainWithTags.csv')
df2 = pd.read_csv('TrumpHillaryTestWithTags.csv')
"""
df2 = pd.read_table('glove.twitter.27B.25d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

def vectorize(w):
  return df2.loc[w].as_matrix()
"""
  

[nltk_data] Downloading package punkt to /home/sasidharan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


'\ndf2 = pd.read_table(\'glove.twitter.27B.25d.txt\', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)\n\ndef vectorize(w):\n  return df2.loc[w].as_matrix()\n'

In [2]:
def handleToNumber(handle):
    if handle == "realDonaldTrump":
        return 0
    else:
        return 1

In [3]:
df1['tokenized_tweets'] = df1.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)
df1['labels'] = df1.apply(lambda row: handleToNumber(row['handle']), axis=1)
df2['tokenized_tweets'] = df2.apply(lambda row: nltk.word_tokenize(row['tweet']), axis=1)

In [4]:
tweets = df1['tokenized_tweets']
tweets_test = df2['tokenized_tweets']

In [5]:
length = []
_ = df1.apply(lambda row: length.append(len(row['tokenized_tweets'])), axis=1)
max_length = max(length)

In [ ]:
max_length

54

In [ ]:
vectorizer = gs.models.Word2Vec(df1['tokenized_tweets'].append(df2['tokenized_tweets']) , min_count=1, size = 20, iter = 2000000)

In [ ]:
train_matrix = []
test_matrix = []

for i, tweet in enumerate(tweets):
    #print(type(string))
    train_matrix.append(vectorizer.wv[tweet])
    
for i, test_tweet in enumerate(tweets_test):
    test_matrix.append(vectorizer.wv[test_tweet])

In [ ]:
# np.array(train_matrix).reshape(len(train_matrix),1)

In [ ]:
train_matrix = np.array(train_matrix)
train_matrix[0]

test_matrix = np.array(test_matrix)
test_matrix[0]

In [ ]:
type(train_matrix)

In [ ]:
train_matrix.shape

In [ ]:
train_matrix[0].shape

In [ ]:
twit = train_matrix[4999]

In [ ]:
def pad(twits):
    """
    Does the padding as many times as required
    """
    l = len(twits)
    d= twits.shape[1]
    padding = np.zeros(d)
    
    for i in range(0,max_length-l):
        twits = np.append(twits,padding)
        
    return twits.reshape(max_length,d)

In [ ]:
def pad_matrix(t_matrix):
    """
    Loops through all the vectorized tweets and calls the pad function on each of them.
    """
    
    padded_matrix =[]
    for i,vectorized_tweet in enumerate(t_matrix):
         padded_matrix.append(pad(vectorized_tweet))
    padded_matrix = np.array(padded_matrix)
    
    return padded_matrix

In [ ]:
padded_train_matrix = pad_matrix(train_matrix)
padded_test_matrix = pad_matrix(test_matrix)

In [ ]:
padded_train_matrix.shape

In [ ]:
def next_batch(num, data, labels):
    '''
    Return a total of `num` random samples and labels. 
    '''
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [ ]:
labels = df1['labels']
labels_vector = []

for i, label in enumerate(labels):
    labels_vector.append([label, 1 - label])
labels_vector = np.array(labels_vector)

In [ ]:
def orthogonal_initializer(scale=1.0):
    def _initializer(shape, dtype=tf.float32, partition_info=None):
        if partition_info is not None:
            ValueError(
                "Do not know what to do with partition_info in BN_LSTMCell")
        flat_shape = (shape[0], np.prod(shape[1:]))
        a = np.random.normal(0.0, 1.0, flat_shape)
        u, _, v = np.linalg.svd(a, full_matrices=False)
        q = u if u.shape == flat_shape else v
        q = q.reshape(shape)
        return tf.constant(scale * q[:shape[0], :shape[1]], dtype=dtype)
    return _initializer


def batch_norm(inputs, name_scope, is_training, epsilon=1e-3, decay=0.99):
    with tf.variable_scope(name_scope):
        size = inputs.get_shape().as_list()[1]

        scale = tf.get_variable(
            'scale', [size], initializer=tf.constant_initializer(0.1))
        offset = tf.get_variable('offset', [size])

        population_mean = tf.get_variable(
            'population_mean', [size],
            initializer=tf.zeros_initializer(), trainable=False)
        population_var = tf.get_variable(
            'population_var', [size],
            initializer=tf.ones_initializer(), trainable=False)
        batch_mean, batch_var = tf.nn.moments(inputs, [0])

        # The following part is based on the implementation of :
        # https://github.com/cooijmanstim/recurrent-batch-normalization
        train_mean_op = tf.assign(
            population_mean,
            population_mean * decay + batch_mean * (1 - decay))
        train_var_op = tf.assign(
            population_var, population_var * decay + batch_var * (1 - decay))

        if is_training is True:
            with tf.control_dependencies([train_mean_op, train_var_op]):
                return tf.nn.batch_normalization(
                    inputs, batch_mean, batch_var, offset, scale, epsilon)
        else:
            return tf.nn.batch_normalization(
                inputs, population_mean, population_var, offset, scale,
                epsilon)



class BN_LSTMCell(RNNCell):
    """LSTM cell with Recurrent Batch Normalization.
    This implementation is based on:
         http://arxiv.org/abs/1603.09025
    This implementation is also based on:
         https://github.com/OlavHN/bnlstm
         https://github.com/nicolas-ivanov/Seq2Seq_Upgrade_TensorFlow
    """

    def __init__(self, num_units, is_training,
                 use_peepholes=False, cell_clip=None,
                 initializer=orthogonal_initializer(),
                 num_proj=None, proj_clip=None,
                 forget_bias=1.0,
                 state_is_tuple=True,
                 activation=tf.tanh):
        """Initialize the parameters for an LSTM cell.
        Args:
          num_units: int, The number of units in the LSTM cell.
          is_training: bool, set True when training.
          use_peepholes: bool, set True to enable diagonal/peephole
            connections.
          cell_clip: (optional) A float value, if provided the cell state
            is clipped by this value prior to the cell output activation.
          initializer: (optional) The initializer to use for the weight
            matrices.
          num_proj: (optional) int, The output dimensionality for
            the projection matrices.  If None, no projection is performed.
          forget_bias: Biases of the forget gate are initialized by default
            to 1 in order to reduce the scale of forgetting at the beginning of
            the training.
          state_is_tuple: If True, accepted and returned states are 2-tuples of
            the `c_state` and `m_state`.  If False, they are concatenated
            along the column axis.
          activation: Activation function of the inner states.
        """
        if not state_is_tuple:
            tf.logging.log_first_n(
                tf.logging.WARN,
                "%s: Using a concatenated state is slower and "
                " will soon be deprecated.  Use state_is_tuple=True.", 1, self)

        self.num_units = num_units
        self.is_training = is_training
        self.use_peepholes = use_peepholes
        self.cell_clip = cell_clip
        self.num_proj = num_proj
        self.proj_clip = proj_clip
        self.initializer = initializer
        self.forget_bias = forget_bias
        self._state_is_tuple = state_is_tuple
        self.activation = activation

        if num_proj:
            self._state_size = (
                LSTMStateTuple(num_units, num_proj)
                if state_is_tuple else num_units + num_proj)
            self._output_size = num_proj
        else:
            self._state_size = (
                LSTMStateTuple(num_units, num_units)
                if state_is_tuple else 2 * num_units)
            self._output_size = num_units

    @property
    def state_size(self):
        return self._state_size

    @property
    def output_size(self):
        return self._output_size

    def __call__(self, inputs, state, scope=None):

        num_proj = self.num_units if self.num_proj is None else self.num_proj

        if self._state_is_tuple:
            (c_prev, h_prev) = state
        else:
            c_prev = tf.slice(state, [0, 0], [-1, self.num_units])
            h_prev = tf.slice(state, [0, self.num_units], [-1, num_proj])

        dtype = inputs.dtype
        input_size = inputs.get_shape().with_rank(2)[1]

        with tf.variable_scope(scope or type(self).__name__):
            if input_size.value is None:
                raise ValueError(
                    "Could not infer input size from inputs.get_shape()[-1]")

            W_xh = tf.get_variable(
                'W_xh',
                [input_size, 4 * self.num_units],
                initializer=self.initializer)
            W_hh = tf.get_variable(
                'W_hh',
                [num_proj, 4 * self.num_units],
                initializer=self.initializer)
            bias = tf.get_variable('B', [4 * self.num_units])

            xh = tf.matmul(inputs, W_xh)
            hh = tf.matmul(h_prev, W_hh)

            bn_xh = batch_norm(xh, 'xh', self.is_training)
            bn_hh = batch_norm(hh, 'hh', self.is_training)

            # i:input gate, j:new input, f:forget gate, o:output gate
            lstm_matrix = tf.nn.bias_add(tf.add(bn_xh, bn_hh), bias)
            i, j, f, o = tf.split(
                value=lstm_matrix, num_or_size_splits=4, axis=1)

            # Diagonal connections
            if self.use_peepholes:
                w_f_diag = tf.get_variable(
                    "W_F_diag", shape=[self.num_units], dtype=dtype)
                w_i_diag = tf.get_variable(
                    "W_I_diag", shape=[self.num_units], dtype=dtype)
                w_o_diag = tf.get_variable(
                    "W_O_diag", shape=[self.num_units], dtype=dtype)

            if self.use_peepholes:
                c = c_prev * tf.sigmoid(f + self.forget_bias +
                                        w_f_diag * c_prev) + \
                    tf.sigmoid(i + w_i_diag * c_prev) * self.activation(j)
            else:
                c = c_prev * tf.sigmoid(f + self.forget_bias) + \
                    tf.sigmoid(i) * self.activation(j)

            if self.cell_clip is not None:
                c = tf.clip_by_value(c, -self.cell_clip, self.cell_clip)

            bn_c = batch_norm(c, 'cell', self.is_training)

            if self.use_peepholes:
                h = tf.sigmoid(o + w_o_diag * c) * self.activation(bn_c)
            else:
                h = tf.sigmoid(o) * self.activation(bn_c)

            if self.num_proj is not None:
                w_proj = tf.get_variable(
                    "W_P", [self.num_units, num_proj], dtype=dtype)

                h = tf.matmul(h, w_proj)
                if self.proj_clip is not None:
                    h = tf.clip_by_value(h, -self.proj_clip, self.proj_clip)

            new_state = (LSTMStateTuple(c, h)
                         if self._state_is_tuple else tf.concat(1, [c, h]))

        return h, new_state

In [ ]:
tf.reset_default_graph()
#define constants
#unrolled through 28 time steps
time_steps=54
#hidden LSTM units
num_units=25
#rows of 28 pixels
n_input=20
#learning rate for adam
learning_rate=0.0001
#mnist is meant to be classified in 10 classes(0-9).
n_classes=2
#size of batch
batch_size=100

#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
start_x=tf.placeholder("float",[None,time_steps,n_input])
x=tf.Variable(tf.zeros([batch_size,time_steps,n_input], dtype=tf.float32),dtype=tf.float32)
x = tf.identity(start_x)

#input label placeholder
y=tf.placeholder("float",[None,n_classes])
train_flag = tf.placeholder(tf.bool)

#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors
input=tf.unstack(x ,time_steps,1)

#defining the network
#lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
lstm_layer = BN_LSTMCell(num_units=num_units, is_training=train_flag, forget_bias=1, use_peepholes=True)
lstm_layer = tf.nn.rnn_cell.DropoutWrapper(lstm_layer, output_keep_prob=0.5)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias
test=tf.nn.softmax(prediction)

#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<8000:
        batch_x,batch_y=next_batch(batch_size, padded_train_matrix, labels_vector)
        sess.run(opt, feed_dict={start_x: batch_x, y: batch_y, train_flag:True})

        if iter %1000==0:
            acc=sess.run(accuracy,feed_dict={start_x:batch_x,y:batch_y,train_flag:True})
            los=sess.run(loss,feed_dict={start_x:batch_x,y:batch_y,train_flag:True})
            pred=sess.run(test, feed_dict={start_x:batch_x,y:batch_y,train_flag:False})
#             print(pred)
# #             print(batch_y)
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
    pred=sess.run(test, feed_dict={start_x:padded_test_matrix, train_flag:False})
    result = np.array(pred, dtype=np.float)
print("Done")

In [ ]:
result.shape
a = np.zeros(shape=(1444,3))
for i, res in enumerate(result):
    a[i] = [i, result[i][1], result[i][0]]

In [ ]:
np.set_printoptions(threshold=np.nan)
np.set_printoptions(suppress=True)
np.savetxt('test.out', a, delimiter=',', fmt='%1.19f')   # X is an array